In [ ]:
μ_min1 = Array{Float64}(undef, 2, 2, 2, 2)
π0 = Array{Float64}(undef, 2 ,2 , 2, 2)
β = 0.8
ν = 1.1

In [ ]:
using Random
Random.seed!(123)

In [ ]:
for i in 1:2
    for j in 1:2
        matrix = rand(2, 2)
        matrix2 = matrix/sum(matrix)
        μ_min1[i, j, :, :] = matrix2
    end
end
μ_min1

In [ ]:
for i in 1:2
    for j in 1:2
        matrix = rand(2, 2)
        matrix2 = matrix/sum(matrix)
        π0[i, j, :, :] = matrix2
    end
end
π0

In [ ]:
κdot = rand(0.95:0.001:1.08, (2, 2, 2, 2))

In [ ]:
Ȧ = rand(0.98:0.001:1.05, (2, 2))

In [ ]:
L_setup_matrix = rand(2, 2)
L0 = L_setup_matrix/sum(L_setup_matrix)

In [ ]:
w0 = rand(0.5:0.001:1.5, (2, 2))

In [ ]:
using Distributions
θ_var = Frechet()
θ_list = rand(θ_var, 2)

In [ ]:
u̇_0t = rand(0.96:0.001:1.04, (2, 2, 2))
# here, u̇_0t is the the initial guess for u̇_0t in both the time periods, where u̇_0t[:, :, 1] is for the first time period
# and u̇_0t[:, :, 2] is for the second time period.

In [ ]:
ẇ = rand(0.96:0.001:1.04, (2, 2))

In [ ]:
μ_0 = Array{Float64}(undef, 2, 2, 2, 2)
L1 = Array{Float64}(undef, 2, 2)
ẇn = Array{Float64}(undef, 2, 2)#This is going to be our updated wage.
Ṗ = Array{Float64}(undef, 2, 2)
π1 = Array{Float64}(undef, 2, 2, 2, 2)
X = Array{Float64}(undef, 2, 2)
α = Array{Float64}(undef, 2)
Xt1 = Array{Float64}(undef, 2, 2)

In [ ]:
while !all(0.99 .* u̇_0t[:, :, 2] .<= u̇_0t[:, :, 1] .<= 1.01 .* u̇_0t[:, :, 2])
    println("NEXT ITERATION STARTS")
    println("u̇_0t at the beginning of the new iteration")
    println(u̇_0t)
    for n in 1:2
        for j in 1:2
            for i in 1:2
                for k in 1:2
                    denom = sum(μ_min1[n, j, m, h]*((u̇_0t[m, h, 2])^(β/ν)) for m in 1:2, h in 1:2)
                    println("denom")
                    println(denom)
                    numer = (μ_min1[n, j, i, k]*((u̇_0t[i, k, 2])^(β/ν)))
                    println("numer")
                    println(numer)
                    μ_0[n, j, i, k] = numer/denom
                end
            end
        end
    end
    println("μ_0")
    println(μ_0)
    for n in 1:2
        for j in 1:2
            L1[n, j] = sum(μ_0[i, k, n, j]*L0[i, k] for i in 1:2, k in 1:2)
        end
    end
    println("L1")
    println(L1)
    println("ẇn before the loop starts")
    println(ẇn)
    println("ẇ before the loop starts")
    println(ẇ)
    while !all(0.99*ẇ .<= ẇn .<= 1.01*ẇ)
        println("inner loop starts")
        for n in 1:2
            for i in 1:2
                for j in 1:2
                    Ṗ[n, j] = (sum(π0[n, j, i, j]*((ẇ[i, j]*κdot[n, j, i, j])^(-θ_list[j]))*((Ȧ[i, j])^θ_list[j]) for i in 1:2))^(-1/θ_list[j])   
                end
            end
        end
        println("Ṗ")
        println(Ṗ)
        for n in 1:2
            for i in 1:2
                for j in 1:2
                    π1[n, j, i, j] = π0[n, j, i, j]*(((ẇ[i, j]*κdot[n, j, i, j])/Ṗ[n, j])^(-θ_list[j]))*((Ȧ[i,j])^θ_list[j])
                end
            end
        end
        println("π1")
        println(π1)
        for i in 1:2
            for j in 1:2
                X[i, j] = w0[i, j]*L0[i, j]
            end
        end
        println("X")
        println(X)
        for i in 1:2
            α[i] = sum(X[k, i] for k in 1:2)/sum(X[j, l] for j in 1:2, l in 1:2)
        end
        println("α")
        println(α)
        for n in 1:2
            for j in 1:2
                Xt1[n, j] = α[j]*sum(ẇ[n, k]*L1[n, k]*w0[n, k] for k in 1:2)
            end
        end
        for n in 1:2
            for j in 1:2
                ẇn[n, j] = sum(π1[i, j, n, j]*Xt1[i, j] for i in 1:2)/(L1[n, j]*w0[n, j])
            end
        end
        println("ẇn at the end of inner loop")
        println(ẇn)
        copy!(ẇ, ẇn)
        println("ẇ reassigned")
        println(ẇ)
        println("inner loop ends")
    end
    println("ẇn after the loop ends")
    println(ẇn)
    println("ẇ after the loop ends")
    println(ẇ)
    u̇_1t = Array{Float64}(undef, 2, 2)#here, u̇_1t is our updated guess for u̇_0t[:, :, 1], that is u̇_0t in the first period.
    for n in 1:2
        for j in 1:2
            u̇_1t[n, j] = ẇ[n, j]*(sum(μ_0[n, j, i, k]*(u̇_0t[i, k, 2])^(β/ν) for i in 1:2, k in 1:2))^(ν)
        end
    end
    println("u̇_0t before the loop")
    println(u̇_0t)
    println("u̇_1t")
    println(u̇_1t)
    # HERE FIRST STORE VALUE OF INITIAL UDOT FROM THE SECOND PERIOD INTO 
    u̇_0t[:, :, 1] .= u̇_1t
    println("u̇_0t updated and here it is after the loop")
    println(u̇_0t)
    println("ITERATION ENDS")
    println("------------------------------------------------------------------------------------------------")
    println("------------------------------------------------------------------------------------------------")
    println("------------------------------------------------------------------------------------------------")
end 

LoadError: InterruptException: